In [ ]:
from typing_extensions import Self
import numpy as np
import os
import joblib
import torch

class QuantileModelPredictor:
    def __init__(self, model_directory):
        # Load the scaler and ensure it outputs float64
        self.scaler = joblib.load(os.path.join(model_directory, 'scaler.pkl'))

        self.weights = []
        self.biases = []

        # Load quantiles and their corresponding model weights and biases
        quantiles = np.load(os.path.join(model_directory, 'quantiles.npy'), allow_pickle=True)
        self.quantiles = quantiles

        for q in self.quantiles:
            model_path = os.path.join(model_directory, f'model_quantile_{q}.pth')
            model_info = torch.load(model_path)
            # Initialize lists to hold weights and biases for the quantile model
            model_weights = []
            model_biases = []

            # Extract weights and biases from model state dictionary and convert them to float64
            for name, param in model_info['state_dict'].items():
                if 'weight' in name:
                    model_weights.append(param.detach().numpy().astype(np.float64))
                elif 'bias' in name:
                    model_biases.append(param.detach().numpy().astype(np.float64))

            self.weights.append(model_weights)
            self.biases.append(model_biases)


    def predict_quantiles(self, inputs):
      inputs = np.array(inputs, dtype=np.float64).reshape(1, -1)
      scaled_inputs = self.scaler.transform(inputs).astype(np.float64)

      manual_outputs = []
      for i in range(len(self.quantiles)):
          x = scaled_inputs
          layers = len(self.weights[i])
          for j in range(layers):
              weights = self.weights[i][j]
              biases = self.biases[i][j]
              x = np.dot(x, weights.T) + biases
              if j < layers - 1:  # Apply ReLU to all but the last layer
                  x = np.maximum(0, x)
          manual_outputs.append(x.flatten()[0])

      return manual_outputs

# Usage example:
model_directory = save_path  # Adjust the path as necessary
predictor = QuantileModelPredictor(model_directory)
inputs = [5000, 1000, 15000, 2000]  # Example inputs
manual_outputs = predictor.predict_quantiles(inputs)
print("qunatiles:", predictor.quantiles)
print("Manual Outputs:", manual_outputs)